In [1]:
import parse_data.preprocess as preprocess
import parse_data.loading as loading
from analysis.response_times import calculate_response_times, plot_response_times
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import globals

In [2]:
# paths
data_folder = '/home/tom/Documents/SWC/data' # desktop Duan Lab
# json_filename = '240913_Yansu_Jerry/2024-09-13_11-53-34_YansuSecondSolo.json' 
json_filename = '240913_Yansu_Jerry/2024-09-13_11-31-00_YansuJerrySocial.json'
# data_folder = r'D:\Users\Tom\OneDrive\PhD\SWC\data' # desktop home
# json_filename = r'first_experiments_2409\240913\2024-09-13_11-23-37_YansuFirstSolo.json'


In [3]:
# Load JSON file into pandas df with collapsed data dictionary and adjustments based on date of recording 
df = loading.loading_pipeline(data_folder, json_filename)

Data is from period before 2024-09-13 11:31:00
Running dataframe through playerinfo_playerposition_conversion.
Loading complete.


In [4]:
# Pre-process data 
df = preprocess.standard_preprocessing(df)

Preprocessing complete.


In [63]:

df['trial_epoch'] = np.nan
df['trial_epoch'] = df['trial_epoch'].astype('object')


epoch_transition_triggers = ['trial start', 'slice onset', globals.SELECTED_TRIGGER_ACTIVATION, 'trial end']
epoch_transition_labels = [globals.TRIAL_STARTED, globals.SLICES_ACTIVE, globals.POST_CHOICE, globals.ITI]
epoch_transition_idxs = [df.index[df['eventDescription'] == trigger] for trigger in epoch_transition_triggers]

for i in range(len(epoch_transition_idxs)):
    df.loc[epoch_transition_idxs[i], 'trial_epoch'] = epoch_transition_labels[i]

# df['trial_epoch]

# df.loc[trial_start_idx, 'trial_epoch'] = 'trial start'

df['trial_epoch'] = df['trial_epoch'].ffill()


In [5]:
def create_trial_epoch_column(df, col_name='trial_epoch'):

    df2 = df.copy()
    
    # create column
    df2[col_name] = np.nan
    # cast from float64 to object dtype to allow including strings without complaining
    df2[col_name] = df2[col_name].astype('object')

    # define the eventDescription triggers that lead to epoch transitions, the indices where these occur,
    # and the labels of the epoch periods
    epoch_transition_triggers = ['trial start', 'slice onset', globals.SELECTED_TRIGGER_ACTIVATION, 'trial end']
    epoch_transition_idxs = [df2.index[df2['eventDescription'] == trigger] for trigger in epoch_transition_triggers]
    epoch_transition_labels = [globals.TRIAL_STARTED, globals.SLICES_ACTIVE, globals.POST_CHOICE, globals.ITI]

    # insert the epoch period label at all indexes where this transition occurs
    # and do this for all epoch periods
    for i in range(len(epoch_transition_idxs)):
        df2.loc[epoch_transition_idxs[i], col_name] = epoch_transition_labels[i]
    # add a pre-trials label to the very beginning of recording
    df2.loc[0, col_name] = globals.PRE_TRIALS

    # forward fill trial epoch to replace all nans and make the labels continuous
    df2[col_name] = df2[col_name].ffill()

    return df2

In [6]:
df = create_trial_epoch_column(df)

In [7]:
df['trial_epoch']

0           pre trials
1           pre trials
2           pre trials
3           pre trials
4           pre trials
             ...      
62175    slices active
62176    slices active
62177    slices active
62178    slices active
62179    slices active
Name: trial_epoch, Length: 62180, dtype: object